In [7]:
import sys
import os
parent = os.path.abspath('..')
sys.path.insert(0, parent)

In [8]:
import keras
import tensorflow as tf
import utils
from collections import defaultdict
from models import selfatt_arg_bilstm
from termcolor import colored

import importlib

Using TensorFlow backend.


In [9]:
def load_dataset(filename):
    pickled_object = utils.pickle_from_file(filename)
    return (pickled_object['embeddings'], pickled_object['mappings'],
            pickled_object['data'], pickled_object['datasets'])

In [10]:
MODELS = {
    '19-08-28-13-02' : {
        'classifier': 'Softmax',
        'LSTM-Size': [32, 32],
        'dropout': [0.1, 0.1],
        'charEmbeddingsSize': 32, 'charEmbeddings': 'None',
        'miniBatchSize': 32, 'earlyStopping': 10,
        'n_heads': 4, 'attention_size': 256
    },
    '19-08-31-20-28': {
        'classifier': 'CRF',
        'LSTM-Size': [64, 64],
        'dropout': [0.5, 0.5],
        'charEmbeddingsSize': 32, 'charEmbeddings': 'lstm',
        'miniBatchSize': 64, 'earlyStopping': 10,
        'n_heads': 4, 'attention_size': 256
    }
}

In [11]:
experiment = '19-08-31-20-28'
classifier_params = MODELS[experiment]

In [12]:
embeddings, mappings, data, datasets = load_dataset(
    '../../data/essays2/essays2_komninos_e.p')

In [13]:
import importlib
importlib.reload(selfatt_arg_bilstm)
self_att_model = selfatt_arg_bilstm.SelfAttArgBiLSTM(classifier_params)
self_att_model.setMappings(mappings, embeddings)
self_att_model.setDataset(datasets, data)

In [14]:
self_att_model.buildModel()

W0831 21:06:44.357831 140160844097344 deprecation_wrapper.py:119] From /home/mteruel/anaconda2/envs/env35/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0831 21:06:44.363073 140160844097344 deprecation_wrapper.py:119] From /home/mteruel/anaconda2/envs/env35/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0831 21:06:44.375999 140160844097344 deprecation_wrapper.py:119] From /home/mteruel/anaconda2/envs/env35/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0831 21:06:44.376763 140160844097344 deprecation_wrapper.py:119] From /home/mteruel/anaconda2/envs/env35/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is 

________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
char_input (InputLayer)                                           (None, None, 25)                            0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
words_input (InputLayer)                                          (None, None)                                0                                                                                     

In [15]:
self_att_model.models['essays2'].load_weights(
    '../../results/essays2/{}/essays_model.h5'.format(experiment))

In [16]:
self_att_model.idx2Labels['essays2'].values()

dict_values(['Premise:Attack', 'Premise:Support', 'O', 'Claim:Support:For', 'Claim:For', 'MajorClaim', 'Claim:Against'])

# Prediction visualization

In [17]:
DEFAULT_LABEL = 'O'

STYLES = defaultdict(lambda: 'magenta')
for label in self_att_model.idx2Labels['essays2'].values():
    if 'premise' in label.lower():
        STYLES[label] = 'green'
    elif 'major' in label.lower():
        STYLES[label] = 'blue'
    else:
        STYLES[label] = 'red'
# STYLES.update({
#     'claim': 'red', 'premise': 'green',
#     'major-claim': 'blue'})

def pretty_print(sentence, labels, styles=None, count_words=True):
    if styles is None:
        styles = defaultdict(lambda: 'green')
    result = ''
    previous_label = DEFAULT_LABEL
    for index, (word, label) in enumerate(zip(sentence, labels)):
        if label != previous_label and label == DEFAULT_LABEL:  # end of block
            result = result.strip() + '} '
        if count_words and (index % 10 == 0):
            result += ' \n({})\t'.format(index)
        if label != previous_label and label != DEFAULT_LABEL:  # new block
            result += '{'

        if label == DEFAULT_LABEL:
            result += word + ' '
        else:
            result += colored(word, styles[label]) + ' '
        previous_label = label
    if previous_label != DEFAULT_LABEL:
        result = result.strip() + '}'
    print(result)

In [18]:
import numpy
from keras.preprocessing.sequence import pad_sequences

def model_predict(self_att_model, model_name, sentences, return_attention=False, batch_size=32):
    """Model probability distribution over set of labels for sentences.

    Args:
        model: a Keras model.
    """
    pred_labels = []
    att_scores = []

    for start in range(0, len(sentences), batch_size):
        end = start + batch_size
        instances = []
        for feature_name in self_att_model.params['featureNames']:
            input_data = pad_sequences(
                [numpy.asarray(instance[feature_name])
                 for instance in sentences[start:end]],
                self_att_model.max_sentece_length)
            instances.append(input_data)

        if not return_attention:
            predictions = self_att_model.models[model_name].predict(instances, verbose=False)
        else:
            predictions, attention = self_att_model.label_and_attention(
                self_att_model.models[model_name], instances)
        predictions = predictions.argmax(axis=-1) #Predict classes
    return predictions, attention

In [19]:
import json
from IPython.core.display import display, HTML, Javascript
import os

def get_predictions(model, model_name, sentence, word_range=None):
    """Predicts the labels and obtain the attention scores for sentence.
    
    Returns
        The list of words in sentence
        The list of labels asigned by the classifier
        The array of attention scores, with shape
            [n_layers, n_heads, words, words]
    """
    # Obtain raw output
    result = model_predict(
        model, 'essays2', [sentence],
        return_attention=True, batch_size=1)
    padded_pred_labels, padded_att_scores = result[0][0], result[1][0]
    # Un-pad sequences
    sentence_lenght = len(sentence['tokens'])
    if sentence_lenght > padded_pred_labels.shape[0]:
        raise NotImplementedError('Sentence is longer than padded result. It was cut')

    if word_range is not None:
        start = word_range[0]
        end = min(word_range[1], sentence_lenght)
    else:
        start, end = 0, sentence_lenght
    predictions = padded_pred_labels[start:end]
    attention = numpy.expand_dims(
        padded_att_scores[:, start:end, start:end], axis=0)

    idx2Label = model.idx2Labels[model_name]
    labels = [idx2Label[tag] for tag in predictions]
    return sentence['raw_tokens'], labels, attention

In [20]:
sentence = data['essays2']['trainMatrix'][1]
words, labels, attention = get_predictions(self_att_model, 'essays2', sentence)
pretty_print(words, labels, styles=STYLES)

(0)	{More people are migrating to other countries than ever before  
(10)	The last 50 years have seen an increasing number of  
(20)	immigrants to other countries} . {People moved due to a  
(30)	number of} reasons , {namely better educations or higher salary  
(40)	jobs} . Some {people thought that they should follow the  
(50)	local customs in order to integrate into their adopted countries  
(60)	’ cultures} . However I strongly believe that {they are  
(70)	able to sustain their cultural identities and doing so help  
(80)	they keep their origin values} . Firstly , {maintaining one  
(90)	’ s cultural identity is a key important rule to  
(100)	help individuals emerge in the new multicultural environments} . {Take  
(110)	Australia for example , immigrants from varieties of nations have  
(120)	a day called multicultural day where people from each country  
(130)	prepare their food and traditional activities for displaying in the  
(140)	public venues} . {Many Australians come this

# Attention visualization

In [21]:
%%javascript
require.config({
  paths: {
      d3: 'https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.17/d3',
      jquery: 'ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [24]:
def build_self_attn_params(sentence, attention_matrix, word_range=(0, 20)):
    """Creates the parameters for show function.

    Returns:
        A dictionary with keys:
            attn [array] with shape [n_layers, n_heads, words, words].
            For self attention, we always have one layer
    """
    start, end = word_range
    attn = attention_matrix[:, :, start:end, start:end]
    return {
        'left_text': sentence['raw_tokens'][start:end],
        'right_text': sentence['raw_tokens'][start:end],
        'attn': attn.tolist(),
        'head_maxs': numpy.amax(attn, axis=(-2, -1)).tolist()
    }

def show(sentence, attention_matrix, word_range=(0, 20)):
    vis_html = """
      <span style="user-select:none">
        Layer: <select id="layer"></select>
      </span>
      <div id='vis2'></div> 
    """

    display(HTML(vis_html))
    vis_js = open(os.path.join('js', 'head_view.js')).read()
    attn_data = build_self_attn_params(
        sentence, attention_matrix, word_range=word_range)
    params = {
        'attention': {'all': attn_data},
        'default_filter': "all",
        'divId': '#vis2'
    }
    display(Javascript('window.params = %s' % json.dumps(params)))
    display(Javascript(vis_js))

In [25]:
show(sentence, attention, word_range=(342, 370))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>